In [1004]:
import pandas as pd
import json
import networkx as nx
from networkx_query import search_nodes, search_edges
import numpy as np

In [1005]:
with open('/Users/theojolliffe/Documents/Wayback BBC/topicScoresOrder.json', 'r') as fp:
    topicScoresOrder = json.load(fp)
topicScoresOrder = {i: {i[0]: 1/(1+i[1]/10) for i in topicScoresOrder[i]} for i in topicScoresOrder}

In [1036]:
bbcReg = pd.read_csv('/Users/theojolliffe/Downloads/bbc_regions.csv')
bbcRegLU = {}
for i in bbcReg.index:
    if bbcReg['areaCode'].iloc[i] in [lad['code'] for lad in lads]:
        bbcRegLU[bbcReg['areaCode'].iloc[i]] = bbcReg['bbc_online'].iloc[i]
bbcRegLU['E06000053'] = 'cornwall'
bbcRegLU['E06000060'] = 'beds_bucks_and_herts'
bbcRegLU['E09000001'] = 'london'
bbcRegLU['E07000150'] = 'northamptonshire'

regions = ['cumbria', 'lancashire', 'merseyside', 'manchester', 'tees', 'tyne_and_wear', 'Humberside', 'york_and_north_yorkshire', 'leeds_and_west_yorkshire', 'lincolnshire', 'south_yorkshire', 'birmingham_and_black_country', 'coventry_and_warwickshire', 'hereford_and_worcester', 'shropshire', 'stoke_and_staffordshire', 'derbyshire', 'leicester', 'northamptonshire', 'nottingham', 'bristol', 'cornwall', 'devon', 'gloucestershire', 'somerset', 'wiltshire', 'beds_bucks_and_herts', 'cambridgeshire', 'essex', 'norfolk', 'suffolk', 'berkshire', 'dorset', 'hampshire', 'oxford', 'kent', 'london', 'surrey', 'sussex']
bbcNames = {
     'derby': 'derbyshire', 
     'hereford and worcester': 'hereford_and_worcester',
     'tyne and wear': 'tyne_and_wear',
     'northampton': 'northamptonshire',
     'liverpool': 'merseyside',
     'coventry and warwickshire': 'coventry_and_warwickshire',
     'humberside': 'Humberside',
     'sheffield and south yorkshire': 'south_yorkshire',
     'hampshire & isle of wight': 'hampshire',
     'stoke and staffordshire': 'stoke_and_staffordshire',
     'york & north yorkshire': 'york_and_north_yorkshire',
     'birmingham and black country': 'birmingham_and_black_country',
     'beds, bucks and herts': 'beds_bucks_and_herts',
     'leeds and west yorkshire': 'leeds_and_west_yorkshire'
}
for i in bbcRegLU.keys():
    bbcRegLU[i] = bbcRegLU[i].lower()
    if bbcRegLU[i] not in regions:
        try:
            bbcRegLU[i] = bbcNames[bbcRegLU[i]]
        except:
            pass

In [1037]:
tfidf = pd.read_csv('/Users/theojolliffe/Documents/Wayback BBC/BBCRegionalTFIDF.csv')
tfidf = tfidf.set_index('Unnamed: 0')
for i in tfidf.columns:
    for j in tfidf[i].index:
        tfidf[i].loc[j] = 1+100*tfidf[i].loc[j]
tfidfLU = {}
for i in regions:
    tfidfLU[i]=tfidf.loc[i].to_dict()


In [1038]:
TFtopicLU = {
    'health': 'health',
    'racial': 'ethnicity',
    'population': 'population',
    'age': 'agemed',
    'work': 'economic',
    'commuter': 'travel',
    'housing': 'tenure'
}
TFtopicRev = dict((TFtopicLU[x],x) for x in TFtopicLU)

In [1039]:
options = pd.read_csv('./csv/lists/places_2020.csv')
areaType = pd.read_csv('/Users/theojolliffe/Documents/Census Data/censusAreaLookup.csv')
typeLookup = {}
for i in areaType.index:
    typeLookup[areaType.iloc[i]['Name']]=areaType.iloc[i]['Group name']

In [1040]:
areasO = []
for i in options["code"]:
    try:
        areasO.append(json.load(open(f'/Users/theojolliffe/Documents/Census Data/census-data-main/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
        print(i)
        pass
    
for area in areasO:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']

E05009289
E05009290
E05009293
E05009294
E05009295
E05009297
E05009298
E05009299
E05009300
E05009301
E05009303
E05009306
E05009307
E05009312
E05011090


In [1041]:
areas = []
for i in options["code"]:
    try:
        areas.append(json.load(open(f'/Users/theojolliffe/Documents/pipeline_2/final/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
#         print(i)
        pass
    
for area in areas:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']

In [1042]:
# Seperate the areas by area type
regions = []
for i in areasO:
    if i['type']=='rgn':
        regions.append(i)
        
lads = []
for i in areasO:
    if i['type']=='lad':
        lads.append(i)
        
countries = []
for i in areasO:
    if (i['type']=='ew')|(i['type']=='ctry'):
        countries.append(i)

In [1043]:
# Find each LAD local and national ranks and put in prioirity order
for thisLad in lads:
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_lads = []
    lad_code = thisLad['parents'][0]['code']
    for i in range(len(lads)):
        if  lad_code == lads[i]['parents'][0]['code']:
            sister_lads.append(lads[i])

    # Loop through the various data variables
    for a in thisLad['data']:
        if a in ['population', 'density', 'agemed']:
            b = 'value'
        else:
            b = 'perc'
        # Create nested object with localised rank
        thisLad['data'][a][b+"_rank_local"] = {}    
        for c in ['2011', 'change']:
            thisLad['data'][a][b+"_rank_local"][c] = {}
            for d in thisLad['data'][a][b][c]:
                vari = thisLad['data'][a][b][c][d]

                # Create sorted list of values from sister areas
                group_values = []
                for lad in sister_lads:
                    group_values.append(lad['data'][a][b][c][d])
                    group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                group_values.sort(reverse=True)

                # Find index of value of area of interest
                varRank = group_values.index(vari) + 1

                # Convert bottom half rankings into negative values
                if varRank>len(group_values)/2:
                    varRank = varRank-len(group_values)-1
                    
                natRank = thisLad['data'][a][b+"_rank"][c][d]
                if natRank > 168:
                    natRank = natRank-336-1

                thisLad['data'][a][b+"_rank_local"][c][d] = varRank

                # Append ranking data to original array
                ranks.append({
                    'label': a+'_'+b+'_'+c+'_'+d, 
                    'locRank': varRank, 
                    'natRank': natRank, 
                    'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: (abs(x['locRank']), -abs(x['value'])))
    thisLad["Priorities"] = ranks

In [ ]:
topics = [{1: 'population', 2: 'value', 3: 'all'}, 
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'health', 2: 'perc', 3: 'good'},
          {1: 'travel', 2: 'perc', 3: 'car_van'},
          {1: 'tenure', 2: 'perc', 3: 'rented_social'},
          {1: 'tenure', 2: 'perc', 3: 'rented_private'},
          {1: 'tenure', 2: 'perc', 3: 'owned'},
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'economic', 2: 'perc', 3: 'employee'},
#           {1: 'religion', 2: 'perc', 3: 'Christian'},
          {1: 'health', 2: 'perc', 3: 'bad'},
          {1: 'ethnicity', 2: 'perc', 3: 'black'},
          {1: 'economic', 2: 'perc', 3: 'student'},
          {1: 'economic', 2: 'perc', 3: 'self-employed'}]

ladCodes = []
for lad in lads:
    ladCodes.append(lad['code'])
    
    
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['natRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['change'][topics[j][3]]    
        df1['localRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['change'][topics[j][3]]    
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        df1['parent'].iloc[i] = lads[i]['parents'][0]['name']
    df = pd.concat([df,df1])
df['natRank'] = np.where(df['natRank'] > 168, df['natRank']-337,df['natRank'])



for topic in topics:
    for lad in lads:
        
#       Find the areas that this area has overtaken
        v2001 = lad['data'][topic[1]][topic[2]]['2001'][topic[3]]
        v2011 = lad['data'][topic[1]][topic[2]]['2011'][topic[3]]
        dfT = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]
        obje = []
        for i in range(dfT.shape[0]):
            obje.append(dfT.iloc[i]['lad'])

        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'][topic[3]] = obje
        
#       Find the area immediatly above or below
        below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]
        above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-2
        if below<0:
            below=below+1
            above=above+1
        reg = lad['parents'][0]['name']
        name_above = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[above]['lad']
        name_below = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[below]['lad']
        if above>0:
            area_above = {'name': name_above,
                         'value': df[(df['lad']==name_above)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        else:
            area_above = 'NaN'
        if below<len(df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])]):
            area_below = {'name': name_below,
                          'value': df[(df['lad']==name_below)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        else:
            area_below = 'NaN'
        
        if 'above_below' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'] = {}

        if topic[3] not in lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]] = {}
        
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["above"] = area_above
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["below"] = area_below
            
        # Add top and bottom three biggest movers for each subject to every area
        df_topic = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]
        ob = {}
        for index, row in df_topic.iterrows():
            ob[row['natRank']]= {row['lad']: row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'][topic[3]]=ob

In [ ]:
# Filter out priority list by subject
for lad in lads:
    subjectList = ["fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "bus", "other", "female", "male"]
    priorities = []
    priorities2011 = []
    for rank in lad['Priorities']:
        s=rank['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        if ((s[2] == "change") & (s[3] not in subjectList)):
            priorities.append(rank)
        if (s[2]!="change"):
            priorities2011.append(rank)
    lad['pri'] = priorities
    lad['pri2011'] = priorities2011

In [ ]:
# Create triple data for areas of closest proximity
geogTriples = []
for lad in lads:
    list1 = lad['bounds'][0]+lad['bounds'][1]
    for lad2 in lads:
        list2 = lad2['bounds'][0]+lad2['bounds'][1]
        listDif = [abs(list1[i]-list2[i]) for i in [0,1,2,3]]
        listDif.sort()
        if (sum(listDif[:3]) < 0.3) & (lad!=lad2):
            geogTriples.append([lad['name'], lad2['name'], ("near", round(sum(listDif[:3]), 2))])


In [ ]:
# storiesRefined

In [ ]:
# storiesRefined = sorted(storiesRefined, key=lambda x: ((abs(x['locRank'])*(1/tfidfLU['manchester'][TFtopicRev[x['label'].split('_')[0]]])), 
#                                                        abs(x['natRank']), -abs(x['value'])))
# storiesRefined

In [ ]:
# topicScoresOrder[bbcRegLU['E09000004']]

In [ ]:
# storiesRefined = sorted(storiesRefined, key=lambda x: ((abs(x['locRank'])*(topicScoresOrder[bbcRegLU[thisLad['code']]][x['label'].split('_')[0]])), 
#                                                        abs(x['natRank']), -abs(x['value'])))
# storiesRefined

In [ ]:
# [(TFtopicRev[x['label'].split('_')[0]], 1/tfidfLU['manchester'][TFtopicRev[x['label'].split('_')[0]]]) for x in storiesRefined]

In [ ]:
for thisLad in lads:

    # Find nearby area of same area type
    nearSimilar = []
    for i in geogTriples:
        try:
            if ((i[0]==thisLad['name'])&(typeLookup[i[1]]==typeLookup[thisLad['name']])):
                nearSimilar.append([i[0], i[1], ('near_similar', i[2][1])])
        except:
            pass

    if len(nearSimilar)==0:
        nearSimilar = [i for i in geogTriples if i[0]==thisLad['name']]

    nearSimilar = sorted(nearSimilar, key=lambda x: x[2][1])


    # Find and refine stories
    stories = []
    stories=stories+[i for i in thisLad['pri'] if abs(i['locRank']) <= 5]
    stories=stories+[i for i in thisLad['pri'] if (abs(i['natRank']) <= 10)&(abs(i['locRank']) > 5)]

    change = sorted(thisLad['pri'], reverse=True, key=lambda x: abs(x['value']))

    bigchange = [i for i in change if abs(i['value'])>4.5]
    for i in range(len(bigchange)):
        if len(stories)<4:
            stories=stories+[bigchange[i]]

    smallchange = sorted([i for i in change if abs(i['value'])<1], reverse=False, key=lambda x: abs(x['value']))
    for i in range(len(smallchange)):
        if len(stories)<5:
            stories=stories+[smallchange[i]]

    stories = sorted(stories, key=lambda x: -abs(x['value']))

    storiesRefined =[]
    notInc = ['density', 'age10yr']
    for i in stories:
        if i['label'].split("_")[0] not in notInc:
            storiesRefined.append(i)
            notInc.append(i['label'].split("_")[0])

    if (thisLad['code'][0]!='W'):
        storiesRefined = sorted(storiesRefined, key=lambda x: (abs(x['locRank'])*(topicScoresOrder[bbcRegLU[thisLad['code']]][x['label'].split('_')[0]])*(abs(x['natRank'])/10)*(5/(1+abs(x['value'])))))
        storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='population_value_change_all')

    
    #Find differences in data
    if (thisLad['parents'][0]['name']=='Wales'):                        
        region=[i for i in countries if i['name']=="Wales"][0]
        country=[i for i in countries if i['name']=="Wales"][0]
    else:
        region=[i for i in regions if i['name']==thisLad['parents'][0]['name']][0]
        country=[i for i in countries if i['name']==thisLad['parents'][1]['name']][0]

    def reg(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return region['data'][s[0]][s[1]][s[2]][s[3]]
    regDiff = [(i['label'], i['value']-reg(i)) for i in change]
    regDiff = sorted(regDiff, reverse=True, key=lambda x: abs(x[1]))

    def cou(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return countries[1]['data'][s[0]][s[1]][s[2]][s[3]]
    couDiff = [(i['label'], i['value']-cou(i)) for i in change]
    couDiff = sorted(couDiff, reverse=True, key=lambda x: abs(x[1]))
    
    try:
        nearSimilarData = {}
        nearSimilarData['name'] = nearSimilar[0][1]
        nearSimilarData['data'] = [lad for lad in lads if lad['name']==nearSimilar[0][1]][0]['data']
    except:
        nearSimilarData = []

    def near(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return nearSimilarData['data'][s[0]][s[1]][s[2]][s[3]]
    if len(nearSimilarData)>0:
        nearDiff = [(i['label'], i['value']-near(i)) for i in change]
        nearDiff = sorted(nearDiff, reverse=True, key=lambda x: abs(x[1]))

    nearTops = sorted([(lad['name'], lad['pri'][0]) for lad in lads if lad['name'] in [i[1] for i in nearSimilar if i[0]==thisLad['name']]], key=lambda x: abs(x[1]['locRank']))

    ageBandChange = sorted([(i, thisLad['data']['age10yr']['value']['2011'][i]-thisLad['data']['age10yr']['value']['2001'][i]) for i in thisLad['data']['age10yr']['value']['2001'].keys()], 
           reverse=True, key=lambda x: x[1])

    equalAgeChange = [lad['name'] for 
     lad in lads if (lad['parents'][0]['name'] == thisLad['parents'][0]['name']) 
     & (lad['data']['agemed']['value']['change']['all']==thisLad['data']['agemed']['value']['change']['all'])]
    len(equalAgeChange)

    # Add data to object
    thisLad['stories'] = storiesRefined
    thisLad['nearSimilar'] = {}
    thisLad['nearSimilar']['triples'] = nearSimilar
    thisLad['nearSimilar']['nearTops'] = nearTops
    thisLad['nearSimilar']['nearTops'] = nearSimilarData
    thisLad['differences'] = {}
    thisLad['differences']['country'] = couDiff
    thisLad['differences']['region'] = regDiff
    thisLad['differences']['near'] = nearDiff
    thisLad['data']['age10yr']['absChange'] = ageBandChange
    thisLad['data']['agemed']['value_rank_local']['equalAgeChange]'] = len(equalAgeChange)

In [971]:
[i for i in regions if i['name']==thisLad['parents'][0]['name']]

[]

In [975]:
thisLad['parents'][0]['name']

'North East'

In [1032]:
[l for l in lads if l['code']=='E07000150'][0]

{'code': 'E07000150',
 'name': 'Corby',
 'type': 'lad',
 'area': 8028.28,
 'count': 336,
 'parents': [{'code': 'E12000004', 'name': 'East Midlands', 'type': 'rgn'},
  {'code': 'E92000001', 'name': 'England', 'type': 'ctry'},
  {'code': 'K04000001', 'name': 'England and Wales', 'type': 'ew'}],
 'bounds': [[-0.80795, 52.45446], [-0.5964, 52.55862]],
 'data': {'population': {'perc': {'2001': {'female': 51.32, 'male': 48.68},
    '2011': {'female': 50.78, 'male': 49.22},
    'change': {'female': -0.54, 'male': 0.54}},
   'perc_rank': {'2001': {'female': 152, 'male': 184},
    '2011': {'female': 199, 'male': 132},
    'change': {'female': 221, 'male': 113}},
   'value': {'2001': {'all': 53174, 'female': 27287, 'male': 25887},
    '2011': {'all': 61255, 'female': 31108, 'male': 30147},
    'change': {'all': 15.2, 'female': 14.0, 'male': 16.46}},
   'value_rank': {'2001': {'all': 328, 'female': 328, 'male': 327},
    '2011': {'all': 323, 'female': 324, 'male': 323},
    'change': {'all': 25, 

In [ ]:
manchester['data']

In [877]:
for lad in lads:
    try:
        with open('/Users/theojolliffe/Documents/census-data-transformed/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(lad, outfile)
        print("Worked: ", lad['name'])

    except:
        print("Failed: ", lad['name'])


Worked:  Hartlepool
Worked:  Middlesbrough
Worked:  Redcar and Cleveland
Worked:  Stockton-on-Tees
Worked:  Darlington
Worked:  Halton
Worked:  Warrington
Worked:  Blackburn with Darwen
Worked:  Blackpool
Worked:  Kingston upon Hull, City of
Worked:  East Riding of Yorkshire
Worked:  North East Lincolnshire
Worked:  North Lincolnshire
Worked:  York
Worked:  Derby
Worked:  Leicester
Worked:  Rutland
Worked:  Nottingham
Worked:  Herefordshire, County of
Worked:  Telford and Wrekin
Worked:  Stoke-on-Trent
Worked:  Bath and North East Somerset
Worked:  Bristol, City of
Worked:  North Somerset
Worked:  South Gloucestershire
Worked:  Plymouth
Worked:  Torbay
Worked:  Swindon
Worked:  Peterborough
Worked:  Luton
Worked:  Southend-on-Sea
Worked:  Thurrock
Worked:  Medway
Worked:  Bracknell Forest
Worked:  West Berkshire
Worked:  Reading
Worked:  Slough
Worked:  Windsor and Maidenhead
Worked:  Wokingham
Worked:  Milton Keynes
Worked:  Brighton and Hove
Worked:  Portsmouth
Worked:  Southampton
W

In [878]:
!git add .
!git commit -m "Add files"
!git push -u origin main

[main b6e5a98] Add files
 195 files changed, 287 insertions(+), 239 deletions(-)
Enumerating objects: 361, done.
Counting objects: 100% (361/361), done.
Delta compression using up to 16 threads
Compressing objects: 100% (199/199), done.
Writing objects: 100% (199/199), 282.48 KiB | 5.65 MiB/s, done.
Total 199 (delta 186), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (186/186), completed with 161 local objects.
To https://github.com/theojolliffe/census-data.git
   a9bfad6..b6e5a98  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [817]:
[i for i in lads if i['name']=='Manchester'][0]['stories']

[{'label': 'population_value_change_all',
  'locRank': 1,
  'natRank': 2,
  'value': 28.08},
 {'label': 'health_perc_change_good',
  'locRank': 1,
  'natRank': 1,
  'value': 15.87},
 {'label': 'tenure_perc_change_rented_social',
  'locRank': -1,
  'natRank': -4,
  'value': -7.6},
 {'label': 'agemed_value_change_all',
  'locRank': -1,
  'natRank': -3,
  'value': -2.0},
 {'label': 'travel_perc_change_train_metro',
  'locRank': -1,
  'natRank': -7,
  'value': -4.93},
 {'label': 'ethnicity_perc_change_black',
  'locRank': 1,
  'natRank': 8,
  'value': 4.13},
 {'label': 'economic_perc_change_student',
  'locRank': 1,
  'natRank': 6,
  'value': 2.67}]

In [717]:
[i for i in lads if i['name']=='Manchester'][0]['stories']

[{'label': 'travel_perc_change_train_metro',
  'locRank': -1,
  'natRank': -7,
  'value': -4.93},
 {'label': 'tenure_perc_change_rented_social',
  'locRank': -1,
  'natRank': -4,
  'value': -7.6},
 {'label': 'health_perc_change_good',
  'locRank': 1,
  'natRank': 1,
  'value': 15.87},
 {'label': 'economic_perc_change_student',
  'locRank': 1,
  'natRank': 6,
  'value': 2.67},
 {'label': 'ethnicity_perc_change_black',
  'locRank': 1,
  'natRank': 8,
  'value': 4.13},
 {'label': 'population_value_change_all',
  'locRank': 1,
  'natRank': 2,
  'value': 28.08},
 {'label': 'agemed_value_change_all',
  'locRank': -1,
  'natRank': -3,
  'value': -2.0}]

In [718]:
topicScoresOrder['manchester']

{'travel': 0.7621064077360695,
 'tenure': 0.7792718330541142,
 'health': 0.7902066737468546,
 'economic': 0.793664815031433,
 'ethnicity': 0.7955981355335854,
 'population': 0.8037250405900891,
 'agemed': 0.8139850041580589}

In [720]:
1*(7/10)*(5/7)*topicRank

1.4000000000000001

In [721]:
(7/10)

0.7

In [723]:
(5/5)

1.0

In [ ]:
storiesRefined = sorted(storiesRefined, key=lambda x: )

[{'label': 'population_value_change_all',
  'locRank': -1,
  'natRank': -20,
  'value': 0.97},
 {'label': 'economic_perc_change_self-employed',
  'locRank': -1,
  'natRank': -30,
  'value': 0.63},
 {'label': 'health_perc_change_bad',
  'locRank': -5,
  'natRank': -122,
  'value': -3.67},
 {'label': 'agemed_value_change_all',
  'locRank': -6,
  'natRank': -107,
  'value': 0.0}]